In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
key=os.getenv("OpenAI_API_KEY")

In [4]:
llm=ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbp", temperature=0.7)

In [5]:
with open("D:\Projects\MCQ-Generator\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [6]:
TEMPLATE="""
Text:{text}
You are and MCQ maker. Given the above text, it is your job to\
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone
Make sure the questions are not repeated and check all the questions to be confirming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [7]:
quiz_gen_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [8]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz",verbose=True)

In [9]:
TEMPLATE2="""
YOU are an expert english grammaraian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
If the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed and change the tone such that it fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert english writer of the above quiz:
"""

In [10]:
evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [11]:
review_chain=LLMChain(llm=llm,prompt=evaluation_prompt,output_key="review",verbose=True)

In [12]:
generate_evaluation=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"],output_variables=["subject","quiz"],verbose=True)

In [13]:
PATH="D:\Projects\MCQ-Generator\data.txt"

In [14]:
with open(PATH,"r") as file:
    TEXT=file.read()

In [15]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON

In [16]:
import json

In [18]:
with get_openai_callback() as cb:
    generate_evaluation(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)

        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative artificial intelligence (generative AI or GenAI[1]) is artificial intelligence capable of generating text, images, or other media, using generative models.[2][3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]

In the early 2020s, advances in transformer-based deep neural networks enabled a number of generative AI systems notable for accepting natural language prompts as input. These include large language model (LLM) chatbots such as ChatGPT, Copilot, Bard, and LLaMA, and text-to-image artificial intelligence art systems such as Stable Diffusion, Midjourney, and DALL-E.[7][8][9]

Generative AI has uses across a wide range of industries, including art, writing, script writing, software development, product design, healthcare, finance, gaming, marketing, and fashion.

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-3.5-turbp` does not exist', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}